In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

from __future__ 는 구버전의 언어를 써야 하지만 최신 버전의 기능을 써야 하는 상황에서 상위 버전의 기능을 쓸 수 있도록 하는 것

In [15]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data 2/names/*.txt'))

import unicodedata
import string


all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# 유니코드 문자열을 ASCII로 변환, https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        
        # 유니코드 문자열 unistr에 대한 정규화 형식(normal form) form을 반환합니다.
        # form의 유효한 값은 ‘NFC’, ‘NFKC’, ‘NFD’ 및 ‘NFKD’ 입니다.
        c for c in unicodedata.normalize('NFD', s)
        
         # chr 문자에 할당된 일반 범주(general category)를 문자열로 반환
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# 각 언어의 이름 목록인 category_lines 사전 생성
category_lines = {}
all_categories = []

# 파일을 읽고 줄 단위로 분리
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data 2/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data 2/names/Czech.txt', 'data 2/names/German.txt', 'data 2/names/Arabic.txt', 'data 2/names/Japanese.txt', 'data 2/names/Chinese.txt', 'data 2/names/Vietnamese.txt', 'data 2/names/Russian.txt', 'data 2/names/French.txt', 'data 2/names/Irish.txt', 'data 2/names/English.txt', 'data 2/names/Spanish.txt', 'data 2/names/Greek.txt', 'data 2/names/Italian.txt', 'data 2/names/Portuguese.txt', 'data 2/names/Scottish.txt', 'data 2/names/Dutch.txt', 'data 2/names/Korean.txt', 'data 2/names/Polish.txt']
Slusarski


In [16]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [19]:
# Tesor로 변경 

'''
.. NOTE::
역자 주:  One-Hot 벡터는 언어를 다룰 때 자주 이용되며,
단어,글자 등을 벡터로 표현 할 때 단어,글자 사이의 상관 관계를 미리 알 수 없을 경우,
One-Hot으로 표현하여 서로 직교한다고 가정하고 학습을 시작합니다.
동일하게 상관 관계를 알 수 없는 다른 데이터의 경우에도 One-Hot 벡터를 활용 할 수 있습니다.
'''


import torch 

# all_letters 로 문자의 주소 찾기 

def letterToindex(letter) :
    return all_letters.find(letter)

# 검증을 위해 한개의 문자를 <1 x n_letters> 텐서로 변환 

def letterToTensor(letter) :
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToindex(letter)] = 1
    return tensor

# 한 줄(이름)을  <line_length x 1 x n_letters>,
# 또는 One-Hot 문자 벡터의 Array로 변경

def lineTOTensor(line) :
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line) :
        tensor[li][0][letterToindex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print()

print(lineTOTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])

torch.Size([5, 1, 57])


### 네트워크 생성 

In [23]:
import torch.nn as nn

class RNN(nn.Module) :
    def __init__(self, input_size, hidden_size, output_size) :
        super(RNN, self).__init__()
        
        # 입력 및 은닉 상태로 작동하는 2개의 선형 계층이며, 출력 다음에 LogSoftmax 계층이 있다.
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim =1 )
        
        
    def forward(self, input, hidden) :
        # cat함수는 concatenate를 해주는 함수
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [24]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [26]:
input = lineTOTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)


tensor([[-2.8291, -2.9430, -2.9542, -2.9520, -2.8892, -2.8520, -2.8655, -2.8552,
         -2.9375, -3.0097, -2.8407, -2.8236, -2.8635, -2.9296, -2.9012, -2.8835,
         -2.7776, -2.9495]], grad_fn=<LogSoftmaxBackward>)
